# Assignment 10
## Andrew Barton

In [ ]:
# imports
import pandas as pd
import numpy as np
from fredapi import Fred
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import random
import nltk
from nltk.tokenize import sent_tokenize
import re
import os
import glob
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2024-11-08 21:04:12.883785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# select * from container_runtime_lab.fomc_schema.

In [ ]:
# nltk.data.path.append('./container_runtime_lab/fomc_schema/nltk')  # Adjust to your stage's access path


In [2]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)  


## Step 0: Background Research
### Define Hawkish and Dovish:
- The terms hawkish and dovish are used to describe different approaches to monetary policy. Hawkish generally refers to tightening monetary policy that prioritizes controlling inflation, and is typically associated with higher or increasing interest rates. While hawkish policy can help slow inflation, it also may slow economic growth by making it more expensive for firms to borrow money, and can lead to increased unemployment. Dovish policy generally refers to loose monetary policy with lower interest rates that aims to stimulate the economy by decreasing firm's cost of borrowing. 
- According to the paper "Trillion Dollar Words: A New Financial Dataset, Task & Market Analysis," hawkish sentences are any sentence that indicates future monetary policy tightening, while dovish sentences indicate the opposite. A third class, neutral, is also introduced to capture sentences which do not fall into either hawkish or dovish category.

sources:
- https://www.investopedia.com/terms/d/dove.asp
- https://www.investopedia.com/terms/h/hawk.asp

In [ ]:
FRED_API_KEY = os.environ.get('FRED_API')
fred = Fred(api_key=FRED_API_KEY)

In [4]:
# 3. Interest Rate Indicator - **Federal Funds Rate (fed):** The fed rate is set by the Federal Open Market Committee (FOMC) and is the interest rate that banks charge other banks for overnight loans. It affects liquidity in general and has significant impact on bond markets, repo markets, and equity markets.
# Fed Funds Rate
fed_data =  fred.get_series('DFF')
fed_df = pd.DataFrame(fed_data,columns=['fed_rate']).reset_index()
fed_df.columns = ['Date','fed_rate']

# fed_df = fed_df.dropna()
# fed_df = fed_df.resample('M',on='Date').last().reset_index()
# fed_df['join'] = fed_df['Date'].dt.to_period('M')
# fed_df = fed_df.drop(columns='Date')
fed_df


,Date,fed_rate
0,1954-07-01,1.13
1,1954-07-02,1.25
2,1954-07-03,1.25
3,1954-07-04,1.25
4,1954-07-05,0.88
...,...,...
25679,2024-10-20,4.83
25680,2024-10-21,4.83
25681,2024-10-22,4.83
25682,2024-10-23,4.83


In [5]:
recession_data = fred.get_series('USREC')

recession_df = pd.DataFrame(recession_data, columns=['is_recession'])
recession_df = recession_df.reset_index().rename(columns={'index':'date'})
recession_df = recession_df[recession_df['date'].dt.year>1995]


In [6]:
recession_df

,date,is_recession
1693,1996-01-01,0.0
1694,1996-02-01,0.0
1695,1996-03-01,0.0
1696,1996-04-01,0.0
1697,1996-05-01,0.0
...,...,...
2033,2024-05-01,0.0
2034,2024-06-01,0.0
2035,2024-07-01,0.0
2036,2024-08-01,0.0


In [223]:
# # code written by me and chatgpt (and converted to plotly in cell below)
# fed_df_filtered = fed_df[fed_df['Date'].dt.year > 1995].set_index('Date')

# recession_periods = recession_df[recession_df['is_recession'] == 1]

# # Identify the start and end dates of each recession
# recession_starts = recession_df[(recession_df['is_recession'] == 1) &
#                                      (recession_df['is_recession'].shift(1) == 0)]['date']
# recession_ends = recession_df[(recession_df['is_recession'] == 1) &
#                                    (recession_df['is_recession'].shift(-1) == 0)]['date']

# # Plot Fed Funds Rate
# plt.figure(figsize=(12, 6))
# plt.plot(fed_df_filtered, label='Fed Funds Rate', color='blue')

# # Highlight recession periods
# for start, end in zip(recession_starts, recession_ends):
#     plt.axvspan(start, end, color='gray', alpha=0.3, label='Recession' if start == recession_starts.iloc[0] else "")

# # Customize plot
# plt.xlabel('Date')
# plt.ylabel('Fed Funds Rate (%)')
# plt.title('Fed Funds Rate')
# plt.legend()
# # plt.grid()
# plt.show()

In [221]:

# Create figure
fig = go.Figure()

# Add Fed Funds Rate trace
fig.add_trace(
    go.Scatter(
        x=fed_df_filtered.index,
        y=fed_df_filtered['fed_rate'],
        name='Fed Funds Rate',
        mode='lines',
        line=dict(color='blue')
    )
)

# Identify the start and end dates of each recession
recession_starts = recession_df[(recession_df['is_recession'] == 1) &
                                (recession_df['is_recession'].shift(1) == 0)]['date']
recession_ends = recession_df[(recession_df['is_recession'] == 1) &
                              (recession_df['is_recession'].shift(-1) == 0)]['date']

# Add shaded rectangles for recession periods
first = True
for start, end in zip(recession_starts, recession_ends):
    fig.add_vrect(
        x0=start,
        x1=end,
        fillcolor="gray",
        opacity=0.3,
        layer="below",
        line_width=0,
        legendgroup='Recession',
        name='Recession' if first else '',
        showlegend=first
    )
    first = False

# Update layout
fig.update_layout(
    title='Fed Funds Rate',
    xaxis_title='Date',
    yaxis_title='Fed Funds Rate (%)',
    template='plotly_white',
    hovermode='x unified',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False, zeroline=True),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.5
    )
)

# Display the figure
fig.show()

### Key Monetary Policy Events (1996-2024)
- The plot above shows that there have been eight eras of monetary policy from 1996 to 2024, with three recessions. In the years leading up to the burst of the dot com bubble in March of 2000 and the coinciding recession, we see a generally hawkish monetary policy, with rates as high as 7.8%. Then, to combat the recession, the Fed took a dovish stance and began lowering interest rates in an attempt to stimulate the economy. After the economy had recovered in 2004, the Fed began hiking rates again in a more hawkish stance. This continued through 2007, until the subprime mortgage crisis caused the Great Recession. The Fed quickly lowered rates to near-zero, where they stayed until 2016. In 2016, the Fed took a hawkish policy and began hiking rates again, until the Covid pandemic sparked a global economic shutdown, causing the Fed to abruptly drop rates to near-zero. After the pandemic, the fed began aggressively hiking rates to combat hyperinflation caused by pandemic-related stimuli. After rates plateaued at 5.33% in 2023, the fed began to take a dovish stance and has began cutting rates.
- It is interesting to note that in the period just before each of the three recessions from 1996-2024, the Fed switched monetary policy from hawkish to dovish. Because this change has just recently happened, many are expecting a recession in the near future. 

## Step 1: Fine-Tuning RoBERTa 
### 0. data prep

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')



Using device: cpu


#### snowflake only:

In [ ]:
# SELECT * FROM CONTAINER_RUNTIME_LAB.FOMC_SCHEMA.FOMC_PRESS_CONFERENCE_DATA_TRAIN

In [ ]:
# SELECT * FROM CONTAINER_RUNTIME_LAB.FOMC_SCHEMA.FOMC_PRESS_CONFERENCE_DATA_TEST

In [11]:
# train_df = cell30.to_pandas()
# test_df = cell31.to_pandas()

In [27]:
train_df = pd.read_excel('./data/lab-manual-mm-train-5768.xlsx')
test_df = pd.read_excel('./data/lab-manual-mm-test-5768.xlsx')

In [28]:
train_df.rename(columns=str.upper,inplace=True)
test_df.rename(columns=str.upper,inplace=True)

### RoBERTa

In [10]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', clean_up_tokenization_spaces=False)


In [29]:
# from chatgpt
class FOMCDataset(Dataset):
    def __init__(self,sentences,labels, tokenizer, max_length= 256):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self,idx):
        sentence = str(self.sentences[idx])
        label = self.labels[idx]


        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',  # As per instructions
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [30]:
train_df = FOMCDataset(sentences = train_df['SENTENCE'].values,
                       labels=train_df['LABEL'].values,
                       tokenizer=tokenizer,
                       max_length=256)
test_df = FOMCDataset(sentences = test_df['SENTENCE'].values,
                       labels=test_df['LABEL'].values,
                       tokenizer=tokenizer,
                       max_length=256)

In [31]:
def create_data_loader(dataset, batch_size):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [32]:
def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    
    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        logits = outputs.logits
        
        _, preds = torch.max(logits, dim=1)
        
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
    return correct_predictions.double() / n_examples, np.mean(losses)


In [33]:

def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            
            loss = outputs.loss
            logits = outputs.logits
            
            _, preds = torch.max(logits, dim=1)
            
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    return correct_predictions.double() / n_examples, np.mean(losses), all_labels, all_preds


In [34]:
learning_rates = [1e-5, 2e-5, 3e-5]
batch_sizes = [4, 8, 16]
epochs_list = [3, 5, 10]


In [ ]:
results = []

for learning_rate in learning_rates:
    for batch_size in batch_sizes:
        for epochs in epochs_list:
            # print(f"\nTraining with learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")

            train_data_loader = create_data_loader(train_df,batch_size)
            test_data_loader = create_data_loader(test_df,batch_size)

            model = RobertaForSequenceClassification.from_pretrained(
                'roberta-base',
                num_labels=3
            )
            model = model.to(device)

            optimizer = AdamW(model.parameters(),lr=learning_rate,no_deprecation_warning=True)
            total_steps = len(train_data_loader)*epochs
            scheduler = get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=0,
                num_training_steps=total_steps
            )

            best_f1 = 0
            
            for epoch in range(epochs):
                # print(f'Epoch {epoch + 1}/{epochs}')
                train_acc, train_loss = train_epoch(
                    model,
                    train_data_loader,
                    optimizer,
                    device,
                    scheduler,
                    len(train_df)
                )
            #     path = f'best_model_lr{learning_rate}_bs{batch_size}_ep{epochs}_state.bin'
            #     torch.save(model.state_dict(),path)
            # model.load_state_dict(torch.load(path))

            test_acc, test_loss, test_labels, test_preds = eval_model(
                model,
                test_data_loader,
                device,
                len(test_df)
            )

            test_f1 = f1_score(test_labels, test_preds, average='weighted')
            test_conf_matrix = confusion_matrix(test_labels, test_preds)
            test_report = classification_report(test_labels,test_preds)

            # print(f'Test F1 Score: {test_f1:.4f}')
            # print('Confusion Matrix:')
            # print(test_conf_matrix)
            # print('Classification Report:')
            # print(test_report)


            results.append({
                'learning_rate': learning_rate,
                'batch_size': batch_size,
                'epochs': epochs,
                'val_f1': best_f1,
                'test_f1': test_f1,
                'confusion_matrix': test_conf_matrix,
                'classification_report': test_report
            })

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training with learning_rate=1e-05, batch_size=4, epochs=3
Epoch 1/3


/Users/andrewbarton/anaconda3/envs/assignment_05/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 2/3
Epoch 3/3
Test F1 Score: 0.6445
Confusion Matrix:
[[48  6 15]
 [ 6 34  9]
 [27 13 56]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.70      0.64        69
           1       0.64      0.69      0.67        49
           2       0.70      0.58      0.64        96

    accuracy                           0.64       214
   macro avg       0.64      0.66      0.65       214
weighted avg       0.65      0.64      0.64       214


Training with learning_rate=1e-05, batch_size=4, epochs=5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/andrewbarton/anaconda3/envs/assignment_05/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Test F1 Score: 0.7198
Confusion Matrix:
[[52  4 13]
 [ 6 37  6]
 [17 14 65]]
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.75      0.72        69
           1       0.67      0.76      0.71        49
           2       0.77      0.68      0.72        96

    accuracy                           0.72       214
   macro avg       0.71      0.73      0.72       214
weighted avg       0.72      0.72      0.72       214


Training with learning_rate=2e-05, batch_size=4, epochs=3


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/andrewbarton/anaconda3/envs/assignment_05/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
Epoch 2/3
Epoch 3/3
Test F1 Score: 0.6991
Confusion Matrix:
[[46  7 16]
 [ 1 43  5]
 [16 19 61]]
Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.67      0.70        69
           1       0.62      0.88      0.73        49
           2       0.74      0.64      0.69        96

    accuracy                           0.70       214
   macro avg       0.70      0.73      0.70       214
weighted avg       0.71      0.70      0.70       214


Training with learning_rate=2e-05, batch_size=4, epochs=5


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/andrewbarton/anaconda3/envs/assignment_05/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [35]:
# class imbalance
pd.Series(test_df.labels).value_counts(normalize=True)


2    0.448598
0    0.322430
1    0.228972
Name: proportion, dtype: float64

In [82]:
# Convert the confusion matrix to a string format
# for result in results:
#     result['confusion_matrix'] = str(result['confusion_matrix'])
    
# results_df = pd.DataFrame(results)
# results_df['learning_rate'] = results_df['learning_rate'].apply(lambda x: f'{x:.1e}')
# results_df.sort_values(by = 'test_f1',ascending=False)

results_df = pd.read_csv('./data/hypertuning_results.csv')

In [83]:
results_df

,learning_rate,batch_size,num_epochs,test_f1
0,0.00001,4,3,0.688324
1,0.00001,4,5,0.700036
2,0.00001,4,10,0.718637
3,0.00001,8,3,0.671645
4,0.00001,8,5,0.705814
5,0.00001,8,10,0.725042
6,0.00001,16,3,0.651905
7,0.00001,16,5,0.679511
8,0.00001,16,10,0.715399
9,0.00002,4,3,0.668445


In [106]:
# code from chatgpt
# Create an interactive 3D scatter plot using Plotly
fig = go.Figure(data=[
    go.Scatter3d(
        x=results_df['learning_rate'],
        y=results_df['batch_size'],
        z=results_df['num_epochs'],
        mode='markers+text',
        marker=dict(
            size=8,
            color=results_df['test_f1'],    # Set color to test_f1 values
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title="Test F1 Score")
        ),
        text=[f"{f1:.4f}" for f1 in results_df['test_f1']],  # Display test_f1 score as text labels
        textposition="top center"
    )
])

# Set plot layout details
fig.update_layout(
    scene=dict(
        xaxis_title="Learning Rate",
        yaxis_title="Batch Size",
        zaxis_title="Number of Epochs"
    ),
    title="Interactive 3D Scatter Plot of Hyperparameters vs Test F1 Score",
    width=800,    # Increase width
    height=600  
)

fig.show()


In [107]:
fig.write_html("./plots/interactive_hyperparameter_plot.html")


### best model from hypertuning
Params:
- learning_rate=2e-05
- batch_size=16
- epochs=5  
#### Test F1 Score: 0.7288 
#### Confusion Matrix: 
*(predicted class on top, actual class on side)*

*(0: dovish, 1: hawkish, 2: neutral)*

||0|1|2|
|-|---|---|---|
|**0**|97 | 11 | 27 | 
|**1**| 14 | 78 | 22 | 
|**2**| 32 | 29 |186|


#### Classification Report:  
|              |precision   | recall  |f1-score   |support  |
|--------------|------------|---------|-----------|---------|
|            0 |      0.68  |    0.72 |     0.70  |     135 |
|            1 |      0.66  |    0.68 |     0.67  |     114 | 
|            2 |      0.79  |    0.75 |     0.77  |     247 | 
|     accuracy |            |         |     0.73  |     496 | 
|    macro avg |      0.71  |    0.72 |     0.71  |     496 | 
| weighted avg |      0.73  |    0.73 |     0.73  |     496 |


- After hypertuning the model I determined that the best parameters are a learning rate of 2e-5, batch size of 16, and 5 epochs. This model had a weighted average F1 score of 0.7288, which is consistent with the findings in the "Trillion Dollar Words" paper. We can see that the model is significatly better at classifying neutral text compared to hawkish or dovish, however it is important to note that the dataset is not perfectly balanced: 49.8% of the dataset is neutral, while 27.22% is dovish and 22.98% is hawkish. This class imbalance is consistent with the differences in precision, recall, and F1 scores for each class. 
- Model performance changes as number of epochs, batch size, and learning rate varies. As the number of epochs increases, model performance tends to reach a maximum at 5 epochs. For lower learning rates, as batch size increases model performance tends to decrease, but for larger learning rates, model preformance increases with batch size. Model performance tends to peak at a learning rate of 2e-5, however model performance with a learning rate of 1e-5 is only 0.0023 lower on average, while the f1 score for learning rate of 3e-5 is 0.1285 lower on average. 


## Step 2: Inference Using the Fine-Tuned Model

### Load meeting data into snowflake

#### snowflake sql

In [ ]:
# CREATE OR REPLACE TABLE fomc_meeting_minutes (
#     meeting_date DATE,
#     content TEXT
# );


In [ ]:
# -- COPY INTO fomc_meeting_minutes
# -- FROM (
# --   SELECT 
# --       TO_DATE(SPLIT_PART(metadata$filename, '.', 0), 'YYYYMMDD') AS meeting_date,  -- Extract date from filename
# --       $1 AS content  -- File content
# --   FROM @TEXT_DATA  -- Reference your stage
# -- )
# -- //FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 0);


In [ ]:
# -- SELECT * FROM CONTAINER_RUNTIME_LAB.FOMC_SCHEMA.FOMC_MEETING_MINUTES

In [ ]:
# data prep
# meeting_df = cell39.to_pandas()
# meeting_df = meeting_df.dropna()


In [47]:


# Replace this with the path to your folder containing the text files
folder_path = './data/meeting_minutes'

# Get a list of all text files in the folder
file_list = glob.glob(os.path.join(folder_path, '*.txt'))

data = []

for file_path in file_list:
    # Extract the filename from the path
    filename = os.path.basename(file_path)
    
    # Extract the date from the filename (remove '.txt' extension)
    date_str = filename.replace('.txt', '')
    
    # Convert the date string to a datetime object
    try:
        date = pd.to_datetime(date_str, format='%Y%m%d')
    except ValueError as e:
        print(f"Error parsing date from filename {filename}: {e}")
        continue  # Skip this file if date parsing fails
    
    # Read the content of the text file
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            transcript = file.read()
    except Exception as e:
        print(f"Error reading file {filename}: {e}")
        continue  # Skip this file if reading fails
    
    # Append the data to the list
    data.append({'MEETING_DATE': date, 'CONTENT': transcript})


In [49]:

meeting_df = pd.DataFrame(data)
meeting_df.sort_values('MEETING_DATE', inplace=True)
meeting_df.reset_index(drop=True, inplace=True)
meeting_df.dropna(inplace=True)


In [50]:
meeting_df

,MEETING_DATE,CONTENT
0,1996-01-30,reported on recent developments in foreign e...
1,1996-03-26,reported on develop- ments in foreign exchang...
2,1996-05-21,reported on developments in foreign exchange ...
3,1996-07-02,reported on recent developments in foreign ex...
4,1996-08-20,reported on recent developments in foreign ex...
...,...,...
225,2024-03-20,The manager turned first to a review of devel...
226,2024-05-01,The manager turned first to a review of deve...
227,2024-06-12,The manager turned first to a review of devel...
228,2024-07-31,. Use of the overnight reverse repo (ON RRP) f...


In [51]:
keywords = [
    # panel a1
    'inflation expectation', 
    'interest rate', 
    'bank rate', 
    'fund rate', 
    'price', 
    'economic activity', 
    'inflation', 
    'employment',
    
    # panel b1
    'unemployment', 
    'growth', 
    'exchange rate', 
    'productivity', 
    'deficit', 
    'demand', 
    'job market', 
    'monetary policy'
]

In [52]:
def has_keyword(x):
    for keyword in keywords:
        if x.find(keyword) !=-1:
            return True
    return False
        
    

In [53]:
def split_into_sentences(text):
    # This pattern matches periods, question marks, or exclamation points followed by a space and a capital letter
    sentence_endings = re.compile(r'(?<=[.!?])\s+(?=[A-Z])')
    sentences = sentence_endings.split(text)
    return sentences


In [56]:
sentences_data = []

for idx, row in meeting_df.iterrows():
    meeting_date = row['MEETING_DATE']
    content = row['CONTENT']
    sentences = nltk.sent_tokenize(content)
    # for snowflake bc nltk doesnt work
    # sentences = split_into_sentences(content)

    for sentence in sentences:
        sentences_data.append({'MEETING_DATE': meeting_date, 'SENTENCE': sentence})

sentences_df = pd.DataFrame(sentences_data)



In [57]:
sentences_df

,MEETING_DATE,SENTENCE
0,1996-01-30,reported on recent developments in foreign e...
1,1996-01-30,He indicated that the swap line drawing by th...
2,1996-01-30,The Committee ratified that transaction by un...
3,1996-01-30,The Manager also reported on recent developmen...
4,1996-01-30,"By unanimous vote, the Committee ratified the..."
...,...,...
47634,2024-09-18,Governor Bowman preferred at this meeting to l...
47635,2024-09-18,She also expressed her concern that the Commit...
47636,2024-09-18,Consistent with the Committee's decision to lo...
47637,2024-09-18,The Board of Governors of the Federal Reserve ...


In [62]:
# filter df
filtered_sentences_df=sentences_df[sentences_df['SENTENCE'].apply(has_keyword)]

In [63]:
filtered_sentences_df

,MEETING_DATE,SENTENCE
5,1996-01-30,The Committee then turned to a discussion of t...
8,1996-01-30,Consumer spending had expanded modestly on ba...
9,1996-01-30,Slower growth in final sales was leading to i...
10,1996-01-30,The demand for labor was still growing at a mo...
11,1996-01-30,The recent data on prices and wages had been ...
...,...,...
47630,2024-09-18,The Committee would be prepared to adjust the ...
47631,2024-09-18,The Committee's assessments will take into acc...
47634,2024-09-18,Governor Bowman preferred at this meeting to l...
47635,2024-09-18,She also expressed her concern that the Commit...


### Retrain best model

In [64]:
# best params
learning_rate = 2e-5
epochs = 5
batch_size = 16

In [66]:
# retrain model with best hyperparams
train_data_loader = create_data_loader(train_df,batch_size)
test_data_loader = create_data_loader(test_df,batch_size)

model = RobertaForSequenceClassification.from_pretrained(
    'roberta-base',
    num_labels=3
)
model = model.to(device)

optimizer = AdamW(model.parameters(),lr=learning_rate,no_deprecation_warning=True)
total_steps = len(train_data_loader)*epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


for epoch in range(epochs):
    # print(f'Epoch {epoch + 1}/{epochs}')
    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )
#     path = f'best_model_lr{learning_rate}_bs{batch_size}_ep{epochs}_state.bin'
#     torch.save(model.state_dict(),path)
# model.load_state_dict(torch.load(path))

test_acc, test_loss, test_labels, test_preds = eval_model(
    model,
    test_data_loader,
    device,
    len(test_df)
)

test_f1 = f1_score(test_labels, test_preds, average='weighted')
test_conf_matrix = confusion_matrix(test_labels, test_preds)
test_report = classification_report(test_labels,test_preds)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
print(test_f1)
print(test_conf_matrix)
print(test_report)

0.7404878974910779
[[56  2 11]
 [ 5 43  1]
 [23 13 60]]
              precision    recall  f1-score   support

           0       0.67      0.81      0.73        69
           1       0.74      0.88      0.80        49
           2       0.83      0.62      0.71        96

    accuracy                           0.74       214
   macro avg       0.75      0.77      0.75       214
weighted avg       0.76      0.74      0.74       214



### make inferences
- *note: inferences were done for all filtered sentences in the given text data*

In [76]:
label_map = {0: 'Dovish', 1: 'Hawkish', 2: 'Neutral'}


In [71]:
class InferenceDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=256):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = str(self.sentences[idx])
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }


In [74]:
inference_dataset = InferenceDataset(
    sentences=filtered_sentences_df['SENTENCE'].values,
    tokenizer=tokenizer,
    max_length=256
)


inference_data_loader = DataLoader(
    inference_dataset,
    batch_size=batch_size,
    shuffle=False
)


In [75]:
def inference(model, data_loader, device):
    model = model.eval()
    predictions = []
    
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)
            _, preds = torch.max(probs, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions


In [77]:
predicted_classes = inference(model, inference_data_loader, device)

# Map predicted classes to labels
predicted_labels = [label_map[pred] for pred in predicted_classes]

# Add predictions to DataFrame
filtered_sentences_df['PREDICTED_LABEL'] = predicted_labels


/var/folders/8t/h9f22ww143b62kr6jkxc7bqc0000gn/T/ipykernel_86896/2362899967.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sentences_df['PREDICTED_LABEL'] = predicted_labels


In [80]:
filtered_sentences_df['PREDICTION'] = predicted_classes

/var/folders/8t/h9f22ww143b62kr6jkxc7bqc0000gn/T/ipykernel_86896/3467434382.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sentences_df['PREDICTION'] = predicted_classes


In [231]:
filtered_sentences_df

,MEETING_DATE,SENTENCE,PREDICTED_LABEL,PREDICTION
5,1996-01-30,The Committee then turned to a discussion of t...,Neutral,2
8,1996-01-30,Consumer spending had expanded modestly on ba...,Neutral,2
9,1996-01-30,Slower growth in final sales was leading to i...,Dovish,0
10,1996-01-30,The demand for labor was still growing at a mo...,Dovish,0
11,1996-01-30,The recent data on prices and wages had been ...,Neutral,2
...,...,...,...,...
47630,2024-09-18,The Committee would be prepared to adjust the ...,Neutral,2
47631,2024-09-18,The Committee's assessments will take into acc...,Neutral,2
47634,2024-09-18,Governor Bowman preferred at this meeting to l...,Dovish,0
47635,2024-09-18,She also expressed her concern that the Commit...,Hawkish,1


In [233]:
filtered_sentences_df.to_csv('./data/inference.csv')

### create a hawkishness measure

In [138]:
hawkishness_df = filtered_sentences_df.groupby(['MEETING_DATE','PREDICTION']).agg({'SENTENCE':'count'}).reset_index()

In [139]:
hawkishness_measure = {}
for date in hawkishness_df['MEETING_DATE'].unique():
    date_df = hawkishness_df[hawkishness_df['MEETING_DATE']==date]
    num_sentences = date_df['SENTENCE'].sum()
    hawkishness = (date_df[date_df['PREDICTION']==1]['SENTENCE'].values[0] - date_df[date_df['PREDICTION']==0]['SENTENCE'].values[0]) /num_sentences
    
    hawkishness_measure[date] = hawkishness

In [143]:
hawkishness_measure_df = pd.DataFrame(list(hawkishness_measure.items()),columns=['MEETING_DATE','hawkishness'])
hawkishness_measure_df = hawkishness_measure_df.sort_values('MEETING_DATE').reset_index(drop=True)


In [144]:
hawkishness_measure_df

,MEETING_DATE,hawkishness
0,1996-01-30,0.000000
1,1996-03-26,0.259740
2,1996-05-21,0.329545
3,1996-07-02,0.233871
4,1996-08-20,0.240000
...,...,...
225,2024-03-20,0.136364
226,2024-05-01,0.183333
227,2024-06-12,0.025862
228,2024-07-31,-0.049505


## Step 3: Analysis of CPI, PPI, and Recession

In [150]:
# CPI
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = pd.DataFrame(cpi_data, columns=['CPI']).reset_index()
cpi_df.columns = ['Date', 'CPI']
cpi_df['cpi_change'] = cpi_df['CPI'].pct_change(periods=12)*100
cpi_df = cpi_df.drop(columns='CPI').dropna()
cpi_df
# cpi_df['CPI'] = cpi_df['CPI'].shift(-1)
# cpi_df = cpi_df.dropna()
# cpi_df['join'] = cpi_df['Date'].dt.to_period('M')
# cpi_df = cpi_df.drop(columns='Date')



,Date,cpi_change
12,1948-01-01,10.242086
13,1948-02-01,9.481961
14,1948-03-01,6.818182
15,1948-04-01,8.272727
16,1948-05-01,9.384966
...,...,...
928,2024-05-01,3.250210
929,2024-06-01,2.975629
930,2024-07-01,2.923566
931,2024-08-01,2.591227


In [151]:
# PPI
ppi_data = fred.get_series('PPIACO')
ppi_df = pd.DataFrame(ppi_data, columns=['PPI']).reset_index()
ppi_df.columns = ['Date', 'PPI']
ppi_df['ppi_change'] = ppi_df['PPI'].pct_change(periods=12)*100
ppi_df = ppi_df.drop(columns='PPI').dropna()
ppi_df



,Date,ppi_change
12,1914-01-01,-2.479339
13,1914-02-01,-1.666667
14,1914-03-01,-2.500000
15,1914-04-01,-2.500000
16,1914-05-01,-2.521008
...,...,...
1336,2024-05-01,0.647692
1337,2024-06-01,0.791775
1338,2024-07-01,1.379636
1339,2024-08-01,-0.851056


In [173]:
merged_df = pd.merge(cpi_df,ppi_df,on='Date',how='inner')
merged_df = pd.merge(merged_df,recession_df, left_on='Date', right_on='date')
merged_df = pd.merge_asof(hawkishness_measure_df,merged_df,left_on='MEETING_DATE',right_on='Date')


In [174]:
merged_df

,MEETING_DATE,hawkishness,Date,cpi_change,ppi_change,date,is_recession
0,1996-01-30,0.000000,1996-01-01,2.790698,2.766477,1996-01-01,0.0
1,1996-03-26,0.259740,1996-03-01,2.843915,2.017756,1996-03-01,0.0
2,1996-05-21,0.329545,1996-05-01,2.827087,2.562050,1996-05-01,0.0
3,1996-07-02,0.233871,1996-07-01,2.883355,2.154828,1996-07-01,0.0
4,1996-08-20,0.240000,1996-08-01,2.812296,2.557954,1996-08-01,0.0
...,...,...,...,...,...,...,...
225,2024-03-20,0.136364,2024-03-01,3.475131,-0.765185,2024-03-01,0.0
226,2024-05-01,0.183333,2024-05-01,3.250210,0.647692,2024-05-01,0.0
227,2024-06-12,0.025862,2024-06-01,2.975629,0.791775,2024-06-01,0.0
228,2024-07-31,-0.049505,2024-07-01,2.923566,1.379636,2024-07-01,0.0


In [230]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=merged_df['Date'],
        y=merged_df['cpi_change'],
        name='CPI Annual % Change',
        mode='lines',
        line=dict(color='red')
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=merged_df['MEETING_DATE'],
        y=merged_df['hawkishness'],
        name='Hawkishness Measure',
        mode='lines',
        line=dict(color='blue')
    ),
    secondary_y=False,
)


# Add Recession Shading
# Identify recession periods
recession_periods = merged_df[merged_df['is_recession'] == 1]

# Identify the start and end dates of each recession
recession_starts = merged_df[(merged_df['is_recession'] == 1) &
                                     (merged_df['is_recession'].shift(1) == 0)]['date']
recession_ends = merged_df[(merged_df['is_recession'] == 1) &
                                   (merged_df['is_recession'].shift(-1) == 0)]['date']

# Add shaded rectangles for recession periods
first = True
for start, end in zip(recession_starts, recession_ends):
    fig.add_vrect(
        x0=start,
        x1=end,
        fillcolor="gray",
        opacity=0.3,
        layer="below",
        line_width=0,
        legendgroup = 'Recession',
        name = 'Recession',
        showlegend = first
    )
    first = False


# for start, end in zip(recession_starts, recession_ends):
#     plt.axvspan(start, end, color='gray', alpha=0.3, label='Recession' if start == recession_starts.iloc[0] else "")




# Update layout
fig.update_layout(
    title=f'CPI Annual % Change vs Hawkishness Measure',
    xaxis_title='Date',
    yaxis_title='Hawkishness Measure',
    yaxis2_title = 'CPI Annual % Change' ,
    # legend_title='Indicators',
    template='plotly_white',
    hovermode='x unified',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False,zeroline = False),
    yaxis2=dict(showgrid=False,zeroline = False),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.5
)
    )



In [229]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=merged_df['Date'],
        y=merged_df['ppi_change'],
        name='PPI Annual % Change',
        mode='lines',
        line=dict(color='red')
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=merged_df['MEETING_DATE'],
        y=merged_df['hawkishness'],
        name='Hawkishness Measure',
        mode='lines',
        line=dict(color='blue')
    ),
    secondary_y=False,
)


# Add Recession Shading
# Identify recession periods
recession_periods = merged_df[merged_df['is_recession'] == 1]

# Identify the start and end dates of each recession
recession_starts = merged_df[(merged_df['is_recession'] == 1) &
                                     (merged_df['is_recession'].shift(1) == 0)]['date']
recession_ends = merged_df[(merged_df['is_recession'] == 1) &
                                   (merged_df['is_recession'].shift(-1) == 0)]['date']

# Add shaded rectangles for recession periods
first = True
for start, end in zip(recession_starts, recession_ends):
    fig.add_vrect(
        x0=start,
        x1=end,
        fillcolor="gray",
        opacity=0.3,
        layer="below",
        line_width=0,
        legendgroup = 'Recession',
        name = 'Recession',
        showlegend = first
    )
    first = False


# for start, end in zip(recession_starts, recession_ends):
#     plt.axvspan(start, end, color='gray', alpha=0.3, label='Recession' if start == recession_starts.iloc[0] else "")




# Update layout
fig.update_layout(
    title=f'PPI Annual % Change vs Hawkishness Measure',
    xaxis_title='Date',
    yaxis_title='Hawkishness Measure',
    yaxis2_title = 'PPI Annual % Change' ,
    # legend_title='Indicators',
    template='plotly_white',
    hovermode='x unified',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False,zeroline = False),
    yaxis2=dict(showgrid=False,zeroline = False),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.5
)
    )



- The plots of annual percent change in CPI and PPI vs the hawkishness measure confirms the findings in the "Trillion Dollar Words" paper because it shows that there is significant correlation between the two inflation measures and the hawkishness measure. We can see that in times of high inflation, such as the period from 2020-2024, the hawkishness measure is also typically high, and in times of low inflation, such as the period from 2008-2009, the hawkishness measure is low. The paper also mentions that we can see how quick the Fed reacts to economic crises and how well it controls inflation by examining the positive and negative spikes in the hawkishness measure and the consecutive change in CPI or PPI over the following months.

In [210]:
fed_df_filtered

,fed_rate
Date,
1996-01-01,4.73
1996-01-02,6.06
1996-01-03,6.93
1996-01-04,5.69
1996-01-05,5.46
...,...
2024-10-20,4.83
2024-10-21,4.83
2024-10-22,4.83


In [228]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=fed_df_filtered.index,
        y=fed_df_filtered['fed_rate'],
        name='Fed Funds Rate',
        mode='lines',
        line=dict(color='red')
    ),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=merged_df['MEETING_DATE'],
        y=merged_df['hawkishness'],
        name='Hawkishness Measure',
        mode='lines',
        line=dict(color='blue')
    ),
    secondary_y=False,
)


# Add Recession Shading
# Identify recession periods
recession_periods = merged_df[merged_df['is_recession'] == 1]

# Identify the start and end dates of each recession
recession_starts = merged_df[(merged_df['is_recession'] == 1) &
                                     (merged_df['is_recession'].shift(1) == 0)]['date']
recession_ends = merged_df[(merged_df['is_recession'] == 1) &
                                   (merged_df['is_recession'].shift(-1) == 0)]['date']

# Add shaded rectangles for recession periods
first = True
for start, end in zip(recession_starts, recession_ends):
    fig.add_vrect(
        x0=start,
        x1=end,
        fillcolor="gray",
        opacity=0.3,
        layer="below",
        line_width=0,
        legendgroup = 'Recession',
        name = 'Recession',
        showlegend = first
    )
    first = False


# for start, end in zip(recession_starts, recession_ends):
#     plt.axvspan(start, end, color='gray', alpha=0.3, label='Recession' if start == recession_starts.iloc[0] else "")




# Update layout
fig.update_layout(
    title=f'Fed Funds Rate vs Hawkishness Measure',
    xaxis_title='Date',
    yaxis_title='Hawkishness Measure',
    yaxis2_title = 'Fed Funds Rate' ,
    # legend_title='Indicators',
    template='plotly_white',
    hovermode='x unified',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False,zeroline = True),
    yaxis2=dict(showgrid=False,zeroline = False),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.5
)
    )



- This plot of the Fed Funds Rate and the Hawkishness measure show that the Hawkishness measure is a leading indicator of the Fed Funds Rate. Every time interest rates begin to rise, there is a spike above zero in the Hawkishness measure in the preceding months, and everytime rates are lowered, the hawkishness measure dips below zero in the preceding months. Though this plot was not included in the "Trillion Dollar Words" paper, I believe that it is the most illustrative of the relationship between the Hawkishness measure and the Fed's monetary policy.